<a href="https://colab.research.google.com/github/Dushanthimadhushika3/FP-Growth-Algorithm/blob/main/FP_Growth_Algorithm(Association).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 71kB/s 
     |████████████████████████████████| 204kB 18.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=5fff9ec8049d1b846200a5a634f76d3e341a7bb6365a86ad07d9b3b4fc3aa956
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("Colab")\
        .config('spark.ui.port', '4050').getOrCreate()

In [ ]:
import pandas as pd
import glob

csv = glob.glob('/content/drive/MyDrive/L4S1/Big Data'+'/*.csv')

# Read the data from the dataset
for f in csv:
  df = pd.read_csv(f)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9


In [ ]:
sc = df[['user_session','brand','user_id','event_type']]
sc = sc.loc[sc['event_type']=='purchase']
sc = sc[['user_session','brand']]
sc = sc.dropna()
print(sc.shape)
sc.head()

sc.to_csv('/content/purchase.csv')

(1119818, 2)


In [ ]:
session_id_col_name = 'user_session'
product_id_col_name = 'brand'
session_id_col_index = -1
product_id_col_index = -1
data = dict()

with open('/content/purchase.csv') as f:
    lis = [line.rstrip().split(',') for line in f]        
    for i, x in enumerate(lis):              
        if i == 0:
            if type(x) is list and session_id_col_name in x and product_id_col_name in x: # column names
                session_id_col_index = x.index(session_id_col_name)
                product_id_col_index = x.index(product_id_col_name)
            else:
                print('Required columns does not exists in given csv file')
                break
        else:
            if x[session_id_col_index] in data:
                data[x[session_id_col_index]] = data[x[session_id_col_index]] + ',' + x[product_id_col_index]
            else:
                data[x[session_id_col_index]] = x[product_id_col_index]


# Create the pandas DataFrame
dataframe = pd.DataFrame(list(data.items()),columns=[session_id_col_name,product_id_col_name])
dataframe.head()
print(dataframe.shape)


(541658, 2)


In [ ]:
# Preprocess the data
records = []
for i in range(0,541658):
  record = []
  for j in range(1,2):
    val = dataframe.values[i, j].split(",")
    for k in val:
      if k not in record:
        record.append(k)
  records.append([i, record])

print(record)
# Create the pandas DataFrame
out = pd.DataFrame(records, columns = ['id', 'items'])
out.head()

['cnd']


,id,items
0,0,"[lianail, f.o.x, bluesky, pole, yoko, masura, ..."
1,1,[runail]
2,2,"[cnd, ingarden, haruyama, beauty-free, irisk, ..."
3,3,[f.o.x]
4,4,[runail]


In [ ]:
# Import the FPGrowth API from PySpark
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SQLContext

# Convert the pandas dataframe into spark dataframe
sqlContext = SQLContext(spark)
sf = spark.createDataFrame(out,["id", "items"])
sf.show()

+---+--------------------+
| id|               items|
+---+--------------------+
|  0|[lianail, f.o.x, ...|
|  1|            [runail]|
|  2|[cnd, ingarden, h...|
|  3|             [f.o.x]|
|  4|            [runail]|
|  5|      [pole, runail]|
|  6|            [lovely]|
|  7|              [keen]|
|  8|          [jessnail]|
|  9|       [irisk, yoko]|
| 10|[pole, limoni, na...|
| 11|             [estel]|
| 12|[lovely, oniq, ha...|
| 13|           [concept]|
| 14|               [cnd]|
| 15|             [thuya]|
| 16|            [runail]|
| 17|          [roubloff]|
| 18|            [masura]|
| 19|           [beautix]|
+---+--------------------+
only showing top 20 rows



In [ ]:
# Create fpGrowth object by instantiating the FPGrowth constructor with the necessary parameters
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.001)

# Fit the dataframe into the fpGrowth object to get the model prepared
model = fpGrowth.fit(sf)
# Display the frequently occuring items
model.freqItemsets.show()

+---------------+-----+
|          items| freq|
+---------------+-----+
|      [lianail]| 7175|
|    [bpw.style]|19482|
|       [zinger]| 9230|
|         [yoko]| 6560|
|          [cnd]|14501|
|       [masura]|42449|
|     [haruyama]| 8012|
|       [kapous]|23087|
|      [bluesky]|10817|
|        [estel]|26533|
|         [pole]|10917|
|        [f.o.x]| 7300|
|         [milv]| 9608|
|     [ingarden]|19950|
|          [pnb]| 6815|
|          [uno]|15787|
|        [irisk]|54302|
|[irisk, runail]| 9127|
|         [oniq]| 8253|
|       [lovely]| 6089|
+---------------+-----+
only showing top 20 rows



In [ ]:
# Display the generated association rules
model.associationRules.show()

+----------+----------+-------------------+------------------+--------------------+
|antecedent|consequent|         confidence|              lift|             support|
+----------+----------+-------------------+------------------+--------------------+
|  [runail]|   [irisk]|0.12888512320835982|1.2856185419836057|0.016850115755698246|
|   [irisk]|  [runail]|0.16807852381127766| 1.285618541983606|0.016850115755698246|
+----------+----------+-------------------+------------------+--------------------+



In [ ]:
# Predicts what the customer is most likely to buy with the items bought in the dataset
model.transform(sf).show(30)

+---+--------------------+----------+
| id|               items|prediction|
+---+--------------------+----------+
|  0|[lianail, f.o.x, ...|        []|
|  1|            [runail]|   [irisk]|
|  2|[cnd, ingarden, h...|        []|
|  3|             [f.o.x]|        []|
|  4|            [runail]|   [irisk]|
|  5|      [pole, runail]|   [irisk]|
|  6|            [lovely]|        []|
|  7|              [keen]|        []|
|  8|          [jessnail]|        []|
|  9|       [irisk, yoko]|  [runail]|
| 10|[pole, limoni, na...|        []|
| 11|             [estel]|        []|
| 12|[lovely, oniq, ha...|  [runail]|
| 13|           [concept]|        []|
| 14|               [cnd]|        []|
| 15|             [thuya]|        []|
| 16|            [runail]|   [irisk]|
| 17|          [roubloff]|        []|
| 18|            [masura]|        []|
| 19|           [beautix]|        []|
| 20|               [uno]|        []|
| 21|            [masura]|        []|
| 22|            [runail]|   [irisk]|
| 23|       